# How much should we prepare for the event sponsorship budget in 2018?

* [Exploring the Data](#Exploring-the-Data)
* [Creating a Visualization](#Creating-a-Visualization)
* [Modeling the Data](#Modeling-the-Data)

## Exploring the Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data/sponsorship-budget.csv')
df.head()

In [ ]:
df.columns = df.columns.str.replace(' ', '_')
df.head()

### Handling Missing Data 

See the package `missingno` at https://github.com/ResidentMario/missingno.

In [ ]:
!pip install missingno

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(df)

Fill in the missing data with some random value.

In [ ]:
df.number_of_employees = df.number_of_employees.fillna(85)

In [ ]:
df.head()

---

## Creating a Visualization

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 6))
sns.regplot('revenue', 'sponsor_budget', data=df, ax=ax[0])
sns.regplot('number_of_employees', 'sponsor_budget', data=df, ax=ax[1]);

---

## Modeling the Data

We'll try with a very simple approach that uses the data in the previous month to predict the budget in the next month as feature. We then use the linear regression model to predict the budget in 2018. Let's have a look at the data again.

In [ ]:
df.head()

### Importing the Model

In [ ]:
from sklearn.linear_model import LinearRegression

### Selecting a Set of Features

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [ ]:
def get_rmse(scores):
    mse_scores = -scores
    rmse_scores = np.sqrt(mse_scores)
    return rmse_scores.mean()

In [ ]:
y = df[df.year >= 2015]['sponsor_budget']

In [ ]:
X = df[df.year <= 2016][['revenue', 'number_of_employees']]
lr = LinearRegression()
scores = cross_val_score(lr, X, y, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

In [ ]:
X = df[df.year <= 2016][['revenue']]
lr = LinearRegression()
scores = cross_val_score(lr, X, y, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

In [ ]:
X = df[df.year <= 2016][['number_of_employees']]
lr = LinearRegression()
scores = cross_val_score(lr, X, y, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

### Normalizing Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

In [ ]:
X = df[df.year <= 2016][['revenue', 'number_of_employees']]
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)

lr = LinearRegression()
scores = cross_val_score(lr, X_scaled, y_scaled, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

In [ ]:
X = df[df.year <= 2016][['revenue']]
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)

lr = LinearRegression()
scores = cross_val_score(lr, X_scaled, y_scaled, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

In [ ]:
X = df[df.year <= 2016][['number_of_employees']]
scaler_x = StandardScaler()
X_scaled = scaler_x.fit_transform(X)

lr = LinearRegression()
scores = cross_val_score(lr, X_scaled, y_scaled, cv=2, scoring='neg_mean_squared_error')
get_rmse(scores)

### Creating a Final Model

In [ ]:
selected_features = ['revenue']
dependent_variable = ['sponsor_budget']

In [ ]:
X = df[df.year <= 2016][selected_features]
y = df[df.year >= 2015][dependent_variable]

In [ ]:
y

In [ ]:
scaler_x = StandardScaler()
scaler_x.fit(X)
X_scaled = scaler_x.transform(X)

scaler_y = StandardScaler()
scaler_y.fit(y)
y_scaled = scaler_y.transform(y)

In [ ]:
lr = LinearRegression()
lr.fit(X_scaled, y_scaled)

In [ ]:
print(f'Coefficients: {lr.coef_}')
print(f'Independent Term: {lr.intercept_}')

### Making a Prediction

In [ ]:
X_test = df[df.year == 2017][selected_features]
X_test.head()

In [ ]:
X_test_scaled = scaler_x.transform(X_test)
X_test_scaled

In [ ]:
results = lr.predict(X_test_scaled)

In [ ]:
print(f"Let's prepare the event sponsorship budget (in 2018) around {scaler_y.inverse_transform(results)} baht!")